In [1]:
#!pip install -U openpyxl

In [2]:
import os
import re
import pandas as pd
import numpy as np
from datetime import date, datetime
from openpyxl import load_workbook
import warnings
warnings.filterwarnings('ignore')

In [3]:
dh_inicio = datetime.today()
print(dh_inicio)

2025-05-26 11:16:47.401799


### CARGA DOS ARQUIVOS DOS CICLOS

In [4]:
# Caminho da pasta dos arquivos
caminho_pasta = r"C:\Projetos\PPSI\BASE\CICLOS-1-2-3-4"

# Lista os arquivos da pasta
arquivos = os.listdir(caminho_pasta)

# Filtra apenas arquivos (ignora subpastas)
arquivos = [f for f in arquivos if os.path.isfile(os.path.join(caminho_pasta, f))]

# Cria um DataFrame com os nomes
df = pd.DataFrame(arquivos, columns=["nome_arquivo"])

# Salva em Excel
#df.to_excel(r"C:\Projetos\PPSI\BASE\CICLOS-1-2-3-4\CICLO_1_2_3_arquivos.xlsx", index=False)

print("Lista de arquivos salva com sucesso!")

Lista de arquivos salva com sucesso!


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nome_arquivo  627 non-null    object
dtypes: object(1)
memory usage: 5.0+ KB


In [6]:
# Salva em Excel
#df.to_excel(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\FONTE_PAINEL\Arquivos_C1_C2_C3_C4.xlsx", sheet_name = "Arquivos_C1_C2_C3_C4", index=False)

### FORM2_3-Diagn-SI_CONSOLIDADO

In [7]:
# Lista para armazenar os DataFrames da aba FORM2.3
dados_form2_3_base = []

# Lista para armazenar o resumo das contagens
resumo_linhas_antes = []

for arquivo in arquivos:
    caminho_arquivo = os.path.join(caminho_pasta, arquivo)
    
    nome_arquivo = os.path.basename(caminho_arquivo)
    total_linhas_arquivo = None
    linhas_totais = 0
    linhas_lidas = 0

    try:
        # 1. Contar linhas totais no Excel
        wb_contagem = load_workbook(filename=caminho_arquivo, read_only=True)
        ws_contagem = wb_contagem['FORM2.3']
        total_linhas_arquivo = ws_contagem.max_row

        # 2. Ler valores da visão macro (K11 e K12)
        wb = load_workbook(caminho_arquivo, data_only=True)
        ws = wb['FORM2.3']

        id_controle_cis8 = ws['C11'].value
        desc_controle_cis8 = ws['D11'].value  

        resp_macro = ws['K11'].value
        desc_macro = ws['K12'].value

        # 3. Ler dados com pandas a partir da 10ª linha (header=9)
        df = pd.read_excel(caminho_arquivo, sheet_name='FORM2.3', 
                            dtype={'ID': object},      
                            keep_default_na=False,              
                            na_filter=False,
                            header=9, engine='openpyxl')
        
        df = df.iloc[:181]   # apenas linhas de 0 a 181
        
        # Substitui strings vazias, espaços, tabs, etc., por NaN
#        df.replace(r'^\s*$', pd.NA, regex=True, inplace=True)
#        df = df.dropna(how='all')

        # Registrar total inicial
        linhas_totais = len(df)
        
        # Registrar total final e linhas excluídas
        linhas_lidas = len(df)

      # 4. Renomear colunas
        novos_nomes = {
#            df.columns[1]: 'FILTRO',
            df.columns[2]: 'ID',
            df.columns[3]: 'ID CIS 8',
            df.columns[4]: 'GRUPO IMPLE',
            df.columns[5]: 'FUNÇÃO NIST CSF',
            df.columns[6]: 'MEDIDA',
            df.columns[7]: 'DESCRIÇÃO DA MEDIDA',
            df.columns[8]: 'RESPOSTA DAS MEDIDAS',
            df.columns[9]: 'Justificativa do porquê da resposta "Não se aplica"',
            df.columns[10]: 'RESPOSTA DA VISAO MACRO DOS CONTROLES'
        }
        df.rename(columns=novos_nomes, inplace=True)
        
        df = df[~df['ID'].isnull() & (df['ID'].astype(str).str.strip() != '')]
        df.reset_index(drop=True, inplace=True)

       
        # Colunas auxiliares
        df['ID CONTROLE CIS 8'] = id_controle_cis8 
        df['DESCRIÇÃO CONTROLE CIS 8'] = desc_controle_cis8
        df['Nome da Origem.2'] = arquivo
        df['Nome da Origem.1'] = df['Nome da Origem.2'].str.extract(r'-(.*?)-')
        df['RESPOSTA DA VISAO MACRO DOS CONTROLES'] = resp_macro
        df['DESCRIÇÃO DA VISAO MACRO DOS CONTROLES_1'] = desc_macro
        df['ID_Controle_Seg'] = df['ID'].astype(str).str[:2]
        df['Ciclo_Entrega'] = df['Nome da Origem.2'].str[:2]
        #df['ORGAO'] = df['Nome da Origem.2'].str.extract(r'(-.*?-)')
        #df['FILTRO'] = df['FILTRO'].astype(float).astype(int).astype(str)
        df['ID_Controle_Seg'] = df['ID_Controle_Seg'].str.replace(r'^\d+\.\s*', '', regex=True)
    
        # normaliza nomes de colunas
        df = df.rename(columns=lambda x: x.strip())
        
        # converte notas para numérico (ficará NaN se não houver número)
        df["RESPOSTA DA VISAO MACRO DOS CONTROLES"] = pd.to_numeric(
        df["RESPOSTA DA VISAO MACRO DOS CONTROLES"], errors="coerce"
       )
        
        # --------- mesmas regras de blocos ---------
        mask_titulo = df["ID"].astype(str).str.match(r"^\d+([.,]0)?$")
        df["grupo"] = mask_titulo.cumsum()
        df["ordem_no_grupo"] = df.groupby("grupo").cumcount()
        
        # 1) títulos → mantêm o número já existente
        # 2) primeiro sub‑controle → recebe o texto (duplicado)
        idx_primeiro_sub = (mask_titulo == False) & (df["ordem_no_grupo"] == 1)
        df.loc[idx_primeiro_sub, "RESPOSTA DA VISAO MACRO DOS CONTROLES"] = ""
       # df.loc[idx_primeiro_sub, "RESPOSTA DA VISAO MACRO DOS CONTROLES"] = df.loc[idx_primeiro_sub, "DESCRIÇÃO DA VISAO MACRO DOS CONTROLES"]       
       
        # 3) demais sub‑controles → ficam vazios
        idx_esvaziar = (mask_titulo == False) & (df["ordem_no_grupo"] > 1)
        df.loc[idx_esvaziar, "RESPOSTA DA VISAO MACRO DOS CONTROLES"] = ""
        
        # 3) Máscara para linhas cujo ID tem até 2 caracteres
        mask = df['ID'].notna() & (df['ID'].astype(str).str.len() <= 2)
              
        # 4) Move o texto e limpa
        df.loc[mask, 'MEDIDA'] = df.loc[mask, 'ID CIS 8']
        df.loc[mask, 'ID CIS 8'] = ''
       
        # 3) Zera Descrição do controle onde Resposta do controle está vazia
        mask = df['RESPOSTA DA VISAO MACRO DOS CONTROLES'] == ''
        df.loc[mask, 'DESCRIÇÃO DA VISAO MACRO DOS CONTROLES_1'] = ''


       # Mapeamento dos valores textuais para ID numérico
        mapa_descr_macro = {
            5: 'O controle atinge seu objetivo, é bem definido, suas medidas são implementadas continuamente por meio de um processo e seu desempenho é mensurado quantitativamente por meio de indicadores.',
            4: 'O controle atinge seu objetivo, é bem definido e suas medidas são implementadas continuamente por meio de um processo decorrente da política formalizada.',
            3: 'O controle atinge seu objetivo de forma muito mais organizada utilizando os recursos organizacionais. Além disso, o controle é formalizado por meio de uma política institucional, específica ou como parte de outra maior.',
            2: 'O controle atinge seu objetivo por meio da aplicação de um conjunto básico, porém completo, de atividades que podem ser caracterizadas como realizadas.',
            1: 'O controle atinge mais ou menos seu objetivo, por meio da aplicação de um conjunto incompleto de atividades que podem ser caracterizadas como iniciais ou intuitivas (pouco organizadas).',
            0: 'Ausência de capacidade para a implementação das medidas do controle, ou desconhecimento sobre o atendimento das medidas.',
            None: 'Controle Não Foi Respondido'
        }
        # renomeia a coluna numérica
        #df = df.rename(columns={"RESPOSTA DA VISAO MACRO DOS CONTROLES": "RESPOSTA_DESCRICAO_MACRO"})
             
        # aplica o mapeamento:
        df["DESCRIÇÃO MACRO"] = df["RESPOSTA DA VISAO MACRO DOS CONTROLES"].map(mapa_descr_macro)
        
        masc_titulo = df["RESPOSTA DA VISAO MACRO DOS CONTROLES"].isna()
        df.loc[masc_titulo, "DESCRIÇÃO MACRO"] = df.loc[masc_titulo, "RESPOSTA DA VISAO MACRO DOS CONTROLES"].map(mapa_descr_macro)

        #df = df.rename(columns={"RESPOSTA DA VISAO MACRO DOS CONTROLES": "RESPOSTA_DESCRICAO_MACRO"})      

        df = df.rename(columns={"DESCRIÇÃO MACRO": "DESCRIÇÃO DA VISAO MACRO DOS CONTROLES"})

#        df["DESCRIÇÃO MACRO"] = df["RESPOSTA DA VISAO MACRO DOS CONTROLES"].map(mapa_descr_macro)
        
        mask = df["GRUPO IMPLE"].str.contains(r"CIS ", na=False)
        df.loc[mask, "GRUPO IMPLE"] = ""
        
        # Seleciona as colunas desejadas
        colunas = [
            'Nome da Origem.1','Nome da Origem.2','Ciclo_Entrega', 'ID', 'GRUPO IMPLE', 'FUNÇÃO NIST CSF', 'MEDIDA', 
            'DESCRIÇÃO DA MEDIDA', 'RESPOSTA DAS MEDIDAS', 'ID_Controle_Seg', 
            'Justificativa do porquê da resposta "Não se aplica"', "RESPOSTA DA VISAO MACRO DOS CONTROLES",
             'DESCRIÇÃO DA VISAO MACRO DOS CONTROLES'
        ]
        df = df[colunas]

        # Adiciona ao consolidado
        dados_form2_3_base.append(df)
        #print(f"✅ Leitura concluída com sucesso: {arquivo}")

    except Exception as e:
        print(f"❌ Erro ao processar '{arquivo}': {e}")

# 5. Registrar contagem no resumo antes da consolidação
    finally:
        resumo_linhas_antes.append({
            'arquivo': nome_arquivo,
            'total_linhas_no_arquivo': total_linhas_arquivo,
            'linhas_totais_lidas': linhas_totais,
            'linhas_lidas': linhas_lidas
        })

# 6. Consolida tudo em um único DataFrame
df_form2_3 = pd.concat(dados_form2_3_base, ignore_index=True)

# 7. DataFrame com o resumo ANTES da consolidação
df_resumo_antes = pd.DataFrame(resumo_linhas_antes)

# 8. DataFrame com o resumo APÓS a consolidação
df_resumo_apos = df_form2_3.groupby('Nome da Origem.2').size().reset_index(name='linhas_lidas')

# Renomear coluna para permitir o merge
df_resumo_apos = df_resumo_apos.rename(columns={'Nome da Origem.2': 'arquivo'})

# Merge dos dois resumos
df_resumo = pd.merge(df_resumo_antes, df_resumo_apos, on='arquivo', how='outer')

# Reorganizar colunas
colunas = [
    'arquivo',
    'total_linhas_no_arquivo',
    'linhas_totais_lidas',
    'linhas_lidas_x',  # Do antes
    'linhas_lidas_y',  # Do após
    'nome_de_origem.2_distintos'
]

# Renomear colunas para algo mais intuitivo
df_resumo.rename(columns={
    'linhas_lidas_x': 'linhas_lidas_antes',
    'linhas_lidas_y': 'linhas_lidas_apos'
}, inplace=True)

# Preencher NaNs com 0 (se houver erros na leitura de arquivos)
df_resumo.fillna(0, inplace=True)


# Exibir total de arquivos distintos processados
total_distintos = df_form2_3['Nome da Origem.2'].nunique()


total_arquivos_lidos = df_resumo_antes['linhas_lidas'].gt(0).sum()
print(f"📥 Total de arquivos lidos com sucesso: {total_arquivos_lidos}")

total_arquivos_consolidados = df_form2_3['Nome da Origem.2'].nunique()
print(f"📦 Total de arquivos únicos no consolidado (Nome da Origem.2): {total_arquivos_consolidados}")

df_resumo['total_arquivos_lidos'] = total_arquivos_lidos
df_resumo['total_arquivos_consolidados'] = total_arquivos_consolidados

# 10. Resumo da execução
total_arquivos = len(arquivos)
sucesso = df_resumo_antes['linhas_lidas'].gt(0).sum()
erros = total_arquivos - sucesso

print("\n📊 Resumo da execução:")
print(f"🗂️ Total de arquivos considerados: {total_arquivos}")
print(f"✅ Arquivos lidos com sucesso: {sucesso}")
print(f"❌ Arquivos com erro: {erros}")
print(f"📥 Total de arquivos lidos com sucesso: {total_arquivos_lidos}")
print(f"📦 Total de arquivos únicos no consolidado (Nome da Origem.2): {total_arquivos_consolidados}")

print("✅ Consolidação concluída com sucesso!")

❌ Erro ao processar 'C1-FUNDAJ-PlanilhaSISP-FUNDAJ.xlsx': expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao processar 'C1-IFCE-IFCE - ferramenta_frameworkpsi_pt-3-basica-sem-macros-ciclo-1.xlsx': expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao processar 'C1-UFAPE-PPSI-UFAPE.xlsx': expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao processar 'C1-UFMA-ferramenta_frameworkpsi_pt-3-basica-sem-macros.xlsx': expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao processar 'C1-UFR-ferramenta_frameworkpsi_pt-3-basica-sem-macros-ciclo-1_FINAL.xlsx': expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao processar 'C2-FUNDAJ-ferramenta_framework_ppsi_v4_ciclo_2_FUNDAJ.xlsx': expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao processar 'C2-UFMA-ferramenta_framework_ppsi_v4_ciclo_2.xlsx': expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao proc

In [8]:
colunas1 = [
            'Nome da Origem.1','Nome da Origem.2','Ciclo_Entrega', 'ID', 'GRUPO IMPLE', 'FUNÇÃO NIST CSF', 'MEDIDA', 
            'DESCRIÇÃO DA MEDIDA', 'RESPOSTA DAS MEDIDAS', 'ID_Controle_Seg', 
            'Justificativa do porquê da resposta "Não se aplica"', "RESPOSTA DA VISAO MACRO DOS CONTROLES",
             'DESCRIÇÃO DA VISAO MACRO DOS CONTROLES'
        ]
df = df_form2_3
for col in colunas1:
    if col in df.columns:
        vazios = df[col].isnull().sum() + (df[col].astype(str).str.strip() == '').sum()
        print(f"{col}: {vazios} vazios")
    else:
        print(f"Coluna '{col}' não encontrada")


Nome da Origem.1: 0 vazios
Nome da Origem.2: 0 vazios
Ciclo_Entrega: 0 vazios
ID: 0 vazios
GRUPO IMPLE: 11070 vazios
FUNÇÃO NIST CSF: 11070 vazios
MEDIDA: 0 vazios
DESCRIÇÃO DA MEDIDA: 11070 vazios
RESPOSTA DAS MEDIDAS: 11066 vazios
ID_Controle_Seg: 47970 vazios
Justificativa do porquê da resposta "Não se aplica": 86888 vazios
RESPOSTA DA VISAO MACRO DOS CONTROLES: 94437 vazios
DESCRIÇÃO DA VISAO MACRO DOS CONTROLES: 94095 vazios


In [9]:
# Salva em Excel
df_resumo.to_excel(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\ANALISE_BASE\df_form2_3_resumo.xlsx", sheet_name = "df_form2_3_resumo", index=False)
df_form2_3.to_excel(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\ANALISE_BASE\df_form2_3.xlsx", sheet_name = "df_form2_3", index=False)

In [10]:
df_form2_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105165 entries, 0 to 105164
Data columns (total 13 columns):
 #   Column                                               Non-Null Count   Dtype 
---  ------                                               --------------   ----- 
 0   Nome da Origem.1                                     105165 non-null  object
 1   Nome da Origem.2                                     105165 non-null  object
 2   Ciclo_Entrega                                        105165 non-null  object
 3   ID                                                   105165 non-null  object
 4   GRUPO IMPLE                                          105165 non-null  object
 5   FUNÇÃO NIST CSF                                      105165 non-null  object
 6   MEDIDA                                               105165 non-null  object
 7   DESCRIÇÃO DA MEDIDA                                  105165 non-null  object
 8   RESPOSTA DAS MEDIDAS                                 105165 non-

In [11]:
#df_form2_3.head()

In [12]:
df_form2_3 = df_form2_3.reset_index(drop=True)

In [13]:
df_consolidado = df_form2_3

In [14]:
df_consolidado['Ciclo_Entrega'] = df_consolidado['Ciclo_Entrega'].astype(str).str.strip().str.upper()

# Filtrar somente ciclos válidos que seguem o padrão "C" seguido de número
ciclos_ordenados = sorted(
    [c for c in df_consolidado['Ciclo_Entrega'].unique() if c.startswith('C') and c[1:].isdigit()],
    key=lambda x: int(x[1:])
)

df_resultado = pd.DataFrame()
chave = ['Nome da Origem.1', 'ID']

for i, ciclo in enumerate(ciclos_ordenados):
    ciclos_ate_agora = ciclos_ordenados[:i+1]

    df_filtrado = df_consolidado[df_consolidado['Ciclo_Entrega'].isin(ciclos_ate_agora)].copy()

    # Mapeia a ordem dos ciclos dinamicamente
    ordem_map = {c: idx for idx, c in enumerate(ciclos_ate_agora)}
    df_filtrado['ordem_ciclo'] = df_filtrado['Ciclo_Entrega'].map(ordem_map)

    # Ordena para pegar o mais recente por grupo
    df_filtrado = (
        df_filtrado
        .sort_values(chave + ['ordem_ciclo'], ascending=[True]*len(chave) + [False])
        .drop_duplicates(subset=chave, keep='first')
    )

    df_filtrado['Ciclo_Contexto'] = ciclo
    df_resultado = pd.concat([df_resultado, df_filtrado.drop(columns='ordem_ciclo')], ignore_index=True)

In [15]:
#df_resultado.head()

In [16]:
#df_resultado.head()

In [17]:
# Substitui "C1" por "Ciclo 1", "C2" por "Ciclo 2", etc.
df_form2_3['Ciclo_Entrega'] = df_form2_3['Ciclo_Entrega'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')

In [18]:
# Salva em Excel
df_form2_3.to_excel(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\FONTE_PAINEL\FORM2_3-Diagn-SI_C.xlsx", sheet_name = "FORM2_3-Diagn-SI_C", index=False)

In [19]:
# Substitui "C1" por "Ciclo 1", "C2" por "Ciclo 2", etc.
df_resultado['Ciclo_Entrega'] = df_resultado['Ciclo_Entrega'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')
df_resultado['Ciclo_Contexto'] = df_resultado['Ciclo_Contexto'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')

In [20]:
# Salva em Excel
df_resultado.to_excel(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\FONTE_PAINEL\FORM2_3-Diagn-SI_Recente.xlsx", sheet_name = "FORM2_3-Diagn-SI_Recente", index=False)

In [21]:
# Salvando o DataFrame em CSV com o separador correto e garantindo que o ID seja tratado como texto
#df_resultado.to_csv(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\FONTE_PAINEL\FORM2_3-Diagn-SI_Recente.csv",
#    sep=';',  # Usando ponto e vírgula como separador (dependendo da configuração regional)
#    index=False,  # Não incluir o índice
#    encoding='utf-8-sig',  # Codificação para garantir compatibilidade com o Excel
#    float_format="%.2f",  # Formato de números com 2 casas decimais
#    quotechar='"',  # Envolver os campos em aspas para garantir que os valores não sejam alterados
#    quoting=1  # Forçar o uso de aspas
#)

In [22]:
dh_termino = datetime.today()
duracao = dh_termino - dh_inicio
print(dh_inicio)
print(dh_termino)
print(duracao)

2025-05-26 11:16:47.401799
2025-05-26 11:40:36.193005
0:23:48.791206


In [23]:
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117819 entries, 0 to 117818
Data columns (total 14 columns):
 #   Column                                               Non-Null Count   Dtype 
---  ------                                               --------------   ----- 
 0   Nome da Origem.1                                     117819 non-null  object
 1   Nome da Origem.2                                     117819 non-null  object
 2   Ciclo_Entrega                                        117819 non-null  object
 3   ID                                                   117819 non-null  object
 4   GRUPO IMPLE                                          117819 non-null  object
 5   FUNÇÃO NIST CSF                                      117819 non-null  object
 6   MEDIDA                                               117819 non-null  object
 7   DESCRIÇÃO DA MEDIDA                                  117819 non-null  object
 8   RESPOSTA DAS MEDIDAS                                 117819 non-

In [24]:
colunas1 = [
            'Nome da Origem.1','Nome da Origem.2','Ciclo_Entrega', 'ID', 'GRUPO IMPLE', 'FUNÇÃO NIST CSF', 'MEDIDA', 
            'DESCRIÇÃO DA MEDIDA', 'RESPOSTA DAS MEDIDAS', 'ID_Controle_Seg', 
            'Justificativa do porquê da resposta "Não se aplica"', "RESPOSTA DA VISAO MACRO DOS CONTROLES",
             'DESCRIÇÃO DA VISAO MACRO DOS CONTROLES'
        ]
df = df_resultado
for col in colunas1:
    if col in df.columns:
        vazios = df[col].isnull().sum() + (df[col].astype(str).str.strip() == '').sum()
        print(f"{col}: {vazios} vazios")
    else:
        print(f"Coluna '{col}' não encontrada")

Nome da Origem.1: 0 vazios
Nome da Origem.2: 0 vazios
Ciclo_Entrega: 0 vazios
ID: 0 vazios
GRUPO IMPLE: 12402 vazios
FUNÇÃO NIST CSF: 12402 vazios
MEDIDA: 0 vazios
DESCRIÇÃO DA MEDIDA: 12402 vazios
RESPOSTA DAS MEDIDAS: 12397 vazios
ID_Controle_Seg: 53742 vazios
Justificativa do porquê da resposta "Não se aplica": 97902 vazios
RESPOSTA DA VISAO MACRO DOS CONTROLES: 105849 vazios
DESCRIÇÃO DA VISAO MACRO DOS CONTROLES: 105417 vazios
